## Vaccination stratification

In this notebook, we look at how to introduce vaccination effects into the model. There is not that much new content here that we haven't covered in previous notebooks. Essentially, this is an example implementation of a model stratification, that was introduced in notebook 03-stratification-introduction.

### Standard preliminaries
Before we get into the code for heterogeneous mixing, let's start off with some of our standard (or "boilerplate") code to get everything set up.


In [ ]:
# pip install the required packages if running in Colab
try:
  import google.colab
  IN_COLAB = True
  %pip install summerepi
except:
  IN_COLAB = False

In [ ]:
# Standard imports, plotting option and constant definition
from datetime import datetime, timedelta
from typing import List, Union
import pandas as pd
import plotly.express as px
import numpy as np

from summer import CompartmentalModel
from summer import Stratification

pd.options.plotting.backend = "plotly"

COVID_BASE_DATE = datetime(2019, 12, 31)
region = "Malaysia"

In [ ]:
# Get a function to access the Malaysia data if running in Colab
if IN_COLAB:
    !wget https://raw.githubusercontent.com/monash-emu/AuTuMN/master/notebooks/capacity_building/malaysia/get_mys_data.py

import get_mys_data

In [ ]:
# ... and use it to get the actual data
df = get_mys_data.fetch_mys_data()
initial_population = get_mys_data.get_initial_population(region)
observations = get_mys_data.get_target_observations(df, region, "cases")

In [ ]:
# Define the model running period and convert to a numeric representation
start_date = datetime(2021, 1, 30)
end_date = start_date + timedelta(days=300)
start_date_int = (start_date - COVID_BASE_DATE).days
end_date_int = (end_date - COVID_BASE_DATE).days

In [ ]:
# Define a target set of observations to compare against our modelled outputs later
notifications_target = observations[start_date: end_date]["cases_new"]

### Define a model

In [ ]:
def build_unstratified_model(parameters: dict) -> CompartmentalModel:
    """
    Create a compartmental model, with the minimal compartmental structure needed to run and produce some sort of 
    meaningful outputs.
    
    Args:
        parameters: Flow parameters
    Returns:
        A compartmental model currently without stratification applied
    """

    model = CompartmentalModel(
        times=(parameters["start_time"], parameters["end_time"]),
        compartments=["S", "E", "I", "R"],
        infectious_compartments=["I"],
        ref_date=COVID_BASE_DATE
    )

    infectious_seed = parameters["infectious_seed"]

    model.set_initial_population(distribution={"S": initial_population - infectious_seed, "E": 0, "I": infectious_seed})
    
    # Susceptible people can get infected
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=parameters["contact_rate"], 
        source="S", 
        dest="E"
    )
    # Expose people transition to infected
    model.add_transition_flow(
        name="progression",
        fractional_rate=parameters["progression_rate"],
        source="E",
        dest="I",
    )

    # Infectious people recover
    model.add_transition_flow(
        name="recovery",
        fractional_rate=parameters["recovery_rate"],
        source="I",
        dest="R",
    )

    # Add an infection-specific death flow to the I compartment
    model.add_death_flow(name="infection_death", death_rate=parameters["death_rate"], source="I")


    # We will also request an output for the 'progression' flow
    model.request_output_for_flow("progressions", "progression")

    # Finally, we request an output for the notifications, assumed to be a fraction of the 'progression' flow
    model.request_function_output(
        name="notifications", 
        func=lambda x: x * parameters['reporting_fraction'],
        sources=["progressions"]
    )

    return model

### Basic vaccine stratification

In [ ]:
from summer import Stratification

def get_vaccine_stratification(
    compartments_to_stratify: List[str], 
    vaccine_params: dict
) -> Stratification:
    """
    Create a summer stratification object that stratifies compartments into
    strata, which are intended to represent vaccine stratifications.
    
    Args:
        compartments_to_stratify: List of the compartments to stratify
        vaccine_params: A dictionary which speicifies the infectiousness and severity reduction through vaccination
    Returns:
        A summer stratification object to represent strain stratification (not yet applied)
    """
    strata = ["vaccinated", "unvaccinated"]
    
    # Create the stratification
    vaccine_strat = Stratification(name="vaccine", strata=strata, compartments=compartments_to_stratify)

    # Create our population split dictionary, whose keys match the strata with 80% vaccinated and 20% unvaccinated
    pop_split = {"vaccinated": 0.8, "unvaccinated": 0.2}

    # Set a population distribution
    vaccine_strat.set_population_split(pop_split)

    # Adjusting the death risk associated with vaccination
    vaccine_strat.set_flow_adjustments(
        "infection_death",
        {
            "unvaccinated": None,
            "vaccinated": vaccine_params["ve_death"],
        }
    )
    
    # Adjust infectiousness levels for vaccinated population
    vaccine_strat.add_infectiousness_adjustments(
        "I",
        {
            "unvaccinated": None,
            "vaccinated": vaccine_params["ve_infectiousness"],
        },
    )

    return vaccine_strat

In [ ]:
def build_vacc_stratified_model(
    parameters: dict,
    compartments_to_stratify:List[str],
    vaccine_params: dict,
) -> CompartmentalModel:
    """
    Get the model object with the vaccination stratification applied to it.
    
    Arguments:
        parameters: A dictionary containing the parameter values to use
        compartments_to_stratify: A list of compartments to which the stratification is applied
        vaccine_params: A dictionary which speicifies the infectiousness and severity reduction with vaccination  
    Returns:
        The model object        
    """
    
    # Get an unstratified model object
    model = build_unstratified_model(parameters)

    # Get and apply the stratification
    vaccine_strat = get_vaccine_stratification(compartments_to_stratify, vaccine_params)
    model.stratify_with(vaccine_strat)

    return model

### Build and run the model

In [ ]:
# Build and run the stratified model with some arbitrary parameters
general_params = {
    "contact_rate": 0.37,
    "progression_rate": 0.2,
    "recovery_rate": 0.2,
    "death_rate": 0.,
    "reporting_fraction": 0.09,
    "start_time": start_date_int,
    "end_time": end_date_int,
    "infectious_seed": 100.
}

vaccine_params = {
    "ve_death": 0.3,
    "ve_infectiousness": 0.85
}

compartments_to_stratify = ["S", "E", "I", "R"]
vacc_strat_model = build_vacc_stratified_model(general_params, compartments_to_stratify, vaccine_params)

In [ ]:
vacc_strat_model.run()

notifications = vacc_strat_model.get_derived_outputs_df()["notifications"]

pd.DataFrame(
    {
        "observed": notifications_target,
        "vaccinated model": notifications
    }
).plot()

### Compare against unvaccinated
Compare the model run with vaccination in place to the one without vaccination.

In [ ]:
vaccine_off_params = {
    "ve_death": 1.,
    "ve_infectiousness": 1.
}
vacc_off_model = build_vacc_stratified_model(general_params, compartments_to_stratify, vaccine_off_params)
vacc_off_model.run()
vacc_off_notifications = vacc_off_model.get_derived_outputs_df()["notifications"]
pd.DataFrame(
    {
        "observed": notifications_target,
        "vaccinated model": notifications,
        "unvaccinated model": vacc_off_notifications,
    }
).plot()